In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## QA Pair 를 생성할 PDF 를 로드합니다.


In [2]:
from unstructured.partition.pdf import partition_pdf


def extract_pdf_elements(filepath):
    """
    PDF 파일에서 이미지, 테이블, 그리고 텍스트 조각을 추출합니다.
    path: 이미지(.jpg)를 저장할 파일 경로
    fname: 파일 이름
    """
    return partition_pdf(
        filename=filepath,
        extract_images_in_pdf=False,  # PDF 내 이미지 추출 활성화
        infer_table_structure=False,  # 테이블 구조 추론 활성화
        chunking_strategy="by_title",  # 제목별로 텍스트 조각화
        max_characters=4000,  # 최대 문자 수
        new_after_n_chars=3800,  # 이 문자 수 이후에 새로운 조각 생성
        combine_text_under_n_chars=2000,  # 이 문자 수 이하의 텍스트는 결합
    )

In [4]:
# PDF 파일 로드
elements = extract_pdf_elements("전세사기기본+예방+지원.pdf")

In [5]:
# 로드한 TEXT 청크 수
len(elements)

68

In [5]:
elements

In [11]:
print(elements[0])

전세사기피해자 지원 및 주거안정에 관한 특별법

주거 지원

신용 회복 지원

금융 지원

➊ 주택의 인도와 주민등록(전입신고)을 마치고 확정일자를 갖춘 경우 *임차권등기를 마친 경우 및 전세권설정이 유효한 경우도 인정➋ 임차보증금이 3억원 이하인 경우 *시·도별 여건 및 피해자의 여건을 고려하여 2억원의 상한범위 내에서 조정 가능➌ 다수의 임차인에게 임차보증금반환채권을 변제받지 못하는 피해가 발생하였거나 발생할 것이 예상되는 경우 - 임대인의 파산 또는 회생절차 개시, 임차주택의 경매 또는 공매 절차의 개시(국세 또는 지방세 체납으로 인한 임차주택이 압류된 경우 포함), 임차인의 집행권원 확보 등➍ 임대인이 임차보증금반환채무를 이행하지 아니할 의도*가 있었다고 의심할 만한 상당한 이유가 있는 경우 *임대인등에 대한 수사 개시, 임대인등의 기망, 보증금을 반환할 능력이 없는 자에 대한 임차주택 소유권 양도 또는 임차보증금을 반환할 능력 없이 다수의 주택을 취득하여 매입 등

01

02

1. 전세사기피해자법 지원대상

2. 지원대상 결정 절차

전세사기피해자등 결정을 위한 4가지 요건

피해자 결정 및 결과(결정문) 송달

신청

접수·조사

지원혜택신청

피해 임차인

광역시·도

국토부 (위원회) → 임차인

임차인 → 관련기관

신청일로부터 30일 이내

안건 상정 후 30일 이내 (15일 연장 가능)

① 결정 신청서 * 작성서식은 시·도 또는 국토부 홈페이지, ‘안심전세포털’에서 내려받거나 접수처에서 수령② 임대차계약서 사본 1부 (재계약서가 있는 경우 재계약서 포함)③ 주민등록표 초본 1부 (신청서 상 행정정보공동이용에 미동의한 신청인만 해당)④ 개인정보 수집 및 이용 동의서 (접수처에서 서식 제공됨)⑤ 임대인의 파산선고 결정문 또는 회생개시 결정문 사본 1부⑥ 경매·공매개시 관련 서류 사본 (경매통지서 또는 최고서, 공매통지서 등) * 다만, 경매통지서 또는 최고서, 공매통지서를 분실한 경우에는 등기사항전부증명서로도 가능⑦ 집행권원 (판결정본, 

## QA Pair 생성


In [7]:
## 준호 프롬프트

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate only questions based on the below query.
You are an Teacher/Professor in {domain}. 
Your task is to provide exactly **{num_questions}** question(s) for an upcoming quiz/examination. 
You are not to provide more or less than this number of questions. 
The question(s) should be diverse in nature across the document. 
The purpose of question(s) is to test the understanding of the students on the context information provided.
You must also provide the answer to each question. The answer should be based on the context information provided only.

Restrict the question(s) to the context information provided only.
QUESTION and ANSWER should be written in Korean. response in JSON format which contains the `question` and `answer`.
DO NOT USE List in JSON format.
ANSWER should be a complete sentence.

#Format:
    ```json
    {{
        "QUESTION": "민법 제317조와 제449조가 해당 사건에서 어떻게 적용되었나요?",
        "ANSWER": "민법 제317조에 따르면 전세권이 소멸한 후 전세권 설정자가 목적물의 인도를 받기 전까지 전세금 반환을 거부할 수 있으며, 이 조항은 전세금 반환 청구권이 전세물 명도 이전에도 양도 가능함을 명시하고 있습니다."
    }},
    {{
        "QUESTION": "2022년 대법원 판결에서 전세금 반환 청구권에 대한 최종 판결은 무엇이었나요?",
        "ANSWER": "대법원은 2022년 7월 15일 판결에서 전세금 반환 청구권이 전세물 명도 이전에도 양도 가능하다고 판결하였으며, 전세권자의 권리 보호를 강화하였습니다."  
    }},
    {{
        "QUESTION": "전세 계약과 관련된 권리 양도는 어떤 경우에 가능한가요?",
        "ANSWER": "전세금 반환 청구권은 전세물 명도 이전에도 양도할 수 있으며, 이는 전세 계약 자체와 분리될 수 있습니다."    
    }},
    {{
        "QUESTION": "2018년 대법원 판결에서 전세권과 관련된 양도 제한 규정은 어떻게 해석되었나요?",
        "ANSWER": "대법원은 2018년 3월 2일 판결에서 전세금 반환 청구권이 전세권 계약과 분리되어 양도 가능하며, 민법 제317조를 근거로 명도 이전에도 양도 가능하다고 해석했습니다." 
    }}
    ```
    """

)

In [10]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
---------------------

{context}

---------------------
Given this context, generate only questions based on the below query.
You are an Teacher/Professor in {domain}. 
Your task is to provide exactly **{num_questions}** question(s) for an upcoming quiz/examination. 
You are not to provide more or less than this number of questions. 
The question(s) should be diverse in nature across the document. 
The purpose of question(s) is to test the understanding of the students on the context information provided.
You must also provide the answer to each question. The answer should be based on the context information provided only.

Restrict the question(s) to the context information provided only.
QUESTION and ANSWER should be written in Korean. response in JSON format which contains the `question` and `answer`.
DO NOT USE List in JSON format.
ANSWER should be a complete sentence.

#Format:
```json
{{
    "QUESTION": "바이든 대통령이 서명한 '안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령'의 주요 목적 중 하나는 무엇입니까?",
    "ANSWER": "바이든 대통령이 서명한 행정명령의 주요 목적은 AI의 안전 마련과 보안 기준 마련을 위함입니다."
}},
{{
    "QUESTION": "메타의 라마2가 오픈소스 모델 중에서 어떤 유형의 작업에서 가장 우수한 성능을 발휘했습니까?",
    "ANSWER": "메타의 라마2는 RAG 없는 질문과 답변 및 긴 형식의 텍스트 생성에서 오픈소스 모델 중 가장 우수한 성능을 발휘했습니다."    
}},
{{
    "QUESTION": "IDC 예측에 따르면 2027년까지 생성 AI 플랫폼과 애플리케이션 시장의 매출은 얼마로 전망되나요?",
    "ANSWER": "IDC 예측에 따르면 2027년까지 생성 AI 플랫폼과 애플리케이션 시장의 매출은 283억 달러로 전망됩니다."    
}}
```
"""
)

In [8]:
import json
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


def custom_json_parser(response):
    json_string = response.content.strip().removeprefix("```json\n").removesuffix("\n```").strip()
    json_string = f'[{json_string}]'
    return json.loads(json_string)

chain = (
    prompt
    | ChatOpenAI(
        model="gpt-4o-2024-08-06",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    | custom_json_parser
)  # 체인을 구성합니다.

qa_pair = []

for element in elements[1:]:
    if element.text:
        qa_pair.extend(
            chain.invoke(
                {"context": element.text, "domain": "AI", "num_questions": "3"}
            )
        )

```json
{
    "QUESTION": "전세사기피해자 지원 정책에서 경·공매 절차 지원의 주요 내용은 무엇인가요?",
    "ANSWER": "경·공매 절차 지원은 피해 주택의 경·공매 유예나 정지를 신청할 수 있으며, 법률상담 및 경매대행 서비스를 제공하고, 피해 임차인에게 우선 매수할 수 있는 권한을 부여하는 것을 포함합니다."
},
{
    "QUESTION": "전세사기피해자에게 제공되는 주거 지원의 형태는 어떤 것들이 있나요?",
    "ANSWER": "전세사기피해자에게 제공되는 주거 지원은 기존 임차주택을 공공임대로 지원하거나 전세임대로 지원하여 계속 거주할 수 있도록 하며, 대체 공공임대로 지원하는 형태가 있습니다."
},
{
    "QUESTION": "전세사기피해자에게 제공되는 금융 지원의 조건 중 하나는 무엇인가요?",
    "ANSWER": "전세사기피해자가 거주 주택을 경락받거나 신규주택을 구입하는 경우, 소득 7천만원 이하, 자산 4.69억원 이하의 조건을 충족하면 주택구입자금 대출을 받을 수 있습니다."
}
``````json
{
    "QUESTION": "전세사기피해자에게 제공되는 긴급복지지원의 종류는 무엇인가요?",
    "ANSWER": "전세사기피해자에게 제공되는 긴급복지지원의 종류로는 생계지원, 의료지원, 주거지원, 교육지원이 있습니다."
},
{
    "QUESTION": "전세사기피해자에게 법률 및 심리 상담 지원은 어떻게 이루어지나요?",
    "ANSWER": "전세사기피해자에게는 소송대리, 상속재산관리인 선임 지원, 심리 상담 및 치료비 지원이 제공되며, 법률전문가 연계 및 수수료 지원이 포함됩니다."
},
{
    "QUESTION": "전세피해자에게 제공되는 신규 저리전세대출의 신청자격 기준은 무엇인가요?",
    "ANSWER": "신규 저리전세대출의 신청자격 기준은 부부합산 총소득 1.3억원 이하, 부부합산 순자산 가액 4.69억원 이하의 무주택자입니다."
}
``````json

In [9]:
qa_pair

[{'QUESTION': '전세사기피해자 지원 정책에서 경·공매 절차 지원의 주요 내용은 무엇인가요?',
  'ANSWER': '경·공매 절차 지원은 피해 주택의 경·공매 유예나 정지를 신청할 수 있으며, 법률상담 및 경매대행 서비스를 제공하고, 피해 임차인에게 우선 매수할 수 있는 권한을 부여하는 것을 포함합니다.'},
 {'QUESTION': '전세사기피해자에게 제공되는 주거 지원의 형태는 어떤 것들이 있나요?',
  'ANSWER': '전세사기피해자에게 제공되는 주거 지원은 기존 임차주택을 공공임대로 지원하거나 전세임대로 지원하여 계속 거주할 수 있도록 하며, 대체 공공임대로 지원하는 형태가 있습니다.'},
 {'QUESTION': '전세사기피해자에게 제공되는 금융 지원의 조건 중 하나는 무엇인가요?',
  'ANSWER': '전세사기피해자가 거주 주택을 경락받거나 신규주택을 구입하는 경우, 소득 7천만원 이하, 자산 4.69억원 이하의 조건을 충족하면 주택구입자금 대출을 받을 수 있습니다.'},
 {'QUESTION': '전세사기피해자에게 제공되는 긴급복지지원의 종류는 무엇인가요?',
  'ANSWER': '전세사기피해자에게 제공되는 긴급복지지원의 종류로는 생계지원, 의료지원, 주거지원, 교육지원이 있습니다.'},
 {'QUESTION': '전세사기피해자에게 법률 및 심리 상담 지원은 어떻게 이루어지나요?',
  'ANSWER': '전세사기피해자에게는 소송대리, 상속재산관리인 선임 지원, 심리 상담 및 치료비 지원이 제공되며, 법률전문가 연계 및 수수료 지원이 포함됩니다.'},
 {'QUESTION': '전세피해자에게 제공되는 신규 저리전세대출의 신청자격 기준은 무엇인가요?',
  'ANSWER': '신규 저리전세대출의 신청자격 기준은 부부합산 총소득 1.3억원 이하, 부부합산 순자산 가액 4.69억원 이하의 무주택자입니다.'},
 {'QUESTION': '신규 임차자금 무이자대출의 신청자격은 무엇인가요?',
  'ANSWER': '신규 임차자금 무이자

In [10]:
# 디버깅을 위한 데이터셋 추가
additional_qa = [
    {
        "QUESTION": "테디노트 유튜브 채널에 대해서 알려주세요.",
        "ANSWER": "테디노트(TeddyNote)는 데이터 분석, 머신러닝, 딥러닝 등의 주제를 다루는 유튜브 채널입니다. 이 채널을 운영하는 이경록님은 데이터 분석과 인공지능에 대한 다양한 강의를 제공하며, 초보자도 쉽게 따라할 수 있도록 친절하게 설명합니다.",
    },
    {
        "QUESTION": "랭체인 관련 튜토리얼은 어디서 찾을 수 있나요?",
        "ANSWER": "테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314",
    },
    {
        "QUESTION": "테디노트 운영자에 대해서 알려주세요",
        "ANSWER": "테디노트(TeddyNote) 운영자는 이경록(Teddy Lee)입니다. 그는 데이터 분석, 머신러닝, 딥러닝 분야에서 활동하는 전문가로, 다양한 교육 및 강의를 통해 지식을 공유하고 있습니다. 이경록님은 여러 기업과 교육기관에서 파이썬, 데이터 분석, 텐서플로우 등 다양한 주제로 강의를 진행해 왔습니다",
    },
]

In [11]:
qa_pair.extend(additional_qa)
qa_pair

[{'QUESTION': '집달관이 주택 명도집행을 강행한 이유는 무엇인가요?',
  'ANSWER': '집달관은 원고와 채무자가 처남매부간이라는 점에 착안하여 원고의 항의를 묵살하고 명도집행을 강행하였습니다.'},
 {'QUESTION': '대법원은 원심판결을 왜 파기하고 환송했나요?',
  'ANSWER': '대법원은 원심판결이 피고의 명도집행이 불법하다는 원고들의 주장사실을 인정할 충분한 자료가 없다고 판단한 것이 채증법칙에 위배되고 강제집행절차에 관한 법리를 오해한 위법이 있다고 보아 원심판결을 파기하고 환송했습니다.'},
 {'QUESTION': '주택 명도집행 시 집달관이 준수해야 할 절차는 무엇인가요?',
  'ANSWER': '주택 명도집행 시 집달관은 채무자로부터 그 점유를 회수하여 채권자에게 이전해야 하며, 이는 민사소송법 제690조 제1항에 규정되어 있습니다.'},
 {'QUESTION': '주택임대차보호법 제3조와 제12조는 해당 사건에서 어떻게 적용되었나요?',
  'ANSWER': '주택임대차보호법 제3조와 제12조는 임차인이 주민등록을 통해 대항력을 갖추었을 경우, 전세권설정등기가 말소되더라도 그 대항력을 상실하지 않는다는 점을 명시하고 있습니다.'},
 {'QUESTION': '피고가 전세권설정등기를 한 이유는 무엇인가요?',
  'ANSWER': '피고는 주택임대차보호법에 의한 임차인의 대항력을 이미 갖추었지만, 자신의 지위를 강화하기 위해 전세권설정등기를 하였다고 주장하였습니다.'},
 {'QUESTION': '대법원이 원심판결을 파기한 이유는 무엇인가요?',
  'ANSWER': '대법원은 원심이 피고의 전세권설정등기와 관련된 이유 및 경위를 충분히 심리하지 않았고, 주택임대차보호법의 법리를 오해하여 판결에 영향을 미쳤다고 판단하여 원심판결을 파기하였습니다.'},
 {'QUESTION': '여객자동차 운수사업법상 노선 여객자동차 운송사업과 전세버스 운송사업의 구별 기준은 무엇인가요?',
  'ANSWER': '여객자동차 운수사

## jsonl 파일로 저장


In [12]:
original_qa = qa_pair

In [13]:
with open("data/jeonse_fraud.jsons", "w") as f:
    json.dumps(original_qa)

In [16]:
import json

with open("data/jeonse_fraud.jsonl", "w", encoding="utf-8") as f:
    original_qa = json.dumps(qa_pair, ensure_ascii=False)
    
# 디버깅을 위한 데이터셋 추가
additional_qa = [
    {
        "QUESTION": "여러분의 이름을 넣어보세요",
        "ANSWER": "답변을 적어보세요...",
    },
    {
        "QUESTION": "랭체인 관련 튜토리얼은 어디서 찾을 수 있나요?",
        "ANSWER": "테디노트의 위키독스 페이지에는 LangChain에 대한 다양한 한국어 튜토리얼이 제공됩니다. 링크: https://wikidocs.net/book/14314",
    },
    {
        "QUESTION": "테디노트 운영자에 대해서 알려주세요",
        "ANSWER": "테디노트(TeddyNote) 운영자는 이경록(Teddy Lee)입니다. 그는 데이터 분석, 머신러닝, 딥러닝 분야에서 활동하는 전문가로, 다양한 교육 및 강의를 통해 지식을 공유하고 있습니다. 이경록님은 여러 기업과 교육기관에서 파이썬, 데이터 분석, 텐서플로우 등 다양한 주제로 강의를 진행해 왔습니다",
    },
]

original_qa = json.loads(original_qa)
original_qa.extend(additional_qa)
original_qa

[{'QUESTION': '전세사기피해자 지원 정책에서 경·공매 절차 지원의 주요 내용은 무엇인가요?',
  'ANSWER': '경·공매 절차 지원은 피해 주택의 경·공매 유예나 정지를 신청할 수 있으며, 법률상담 및 경매대행 서비스를 제공하고, 피해 임차인에게 우선 매수할 수 있는 권한을 부여하는 것을 포함합니다.'},
 {'QUESTION': '전세사기피해자에게 제공되는 주거 지원의 형태는 어떤 것들이 있나요?',
  'ANSWER': '전세사기피해자에게 제공되는 주거 지원은 기존 임차주택을 공공임대로 지원하거나 전세임대로 지원하여 계속 거주할 수 있도록 하며, 대체 공공임대로 지원하는 형태가 있습니다.'},
 {'QUESTION': '전세사기피해자에게 제공되는 금융 지원의 조건 중 하나는 무엇인가요?',
  'ANSWER': '전세사기피해자가 거주 주택을 경락받거나 신규주택을 구입하는 경우, 소득 7천만원 이하, 자산 4.69억원 이하의 조건을 충족하면 주택구입자금 대출을 받을 수 있습니다.'},
 {'QUESTION': '전세사기피해자에게 제공되는 긴급복지지원의 종류는 무엇인가요?',
  'ANSWER': '전세사기피해자에게 제공되는 긴급복지지원의 종류로는 생계지원, 의료지원, 주거지원, 교육지원이 있습니다.'},
 {'QUESTION': '전세사기피해자에게 법률 및 심리 상담 지원은 어떻게 이루어지나요?',
  'ANSWER': '전세사기피해자에게는 소송대리, 상속재산관리인 선임 지원, 심리 상담 및 치료비 지원이 제공되며, 법률전문가 연계 및 수수료 지원이 포함됩니다.'},
 {'QUESTION': '전세피해자에게 제공되는 신규 저리전세대출의 신청자격 기준은 무엇인가요?',
  'ANSWER': '신규 저리전세대출의 신청자격 기준은 부부합산 총소득 1.3억원 이하, 부부합산 순자산 가액 4.69억원 이하의 무주택자입니다.'},
 {'QUESTION': '신규 임차자금 무이자대출의 신청자격은 무엇인가요?',
  'ANSWER': '신규 임차자금 무이자

In [25]:
original_qa.

[{'QUESTION': '집달관이 주택 명도집행을 강행한 이유는 무엇인가요?',
  'ANSWER': '집달관은 원고와 채무자가 처남매부간이라는 점에 착안하여 원고의 항의를 묵살하고 명도집행을 강행하였습니다.'},
 {'QUESTION': '대법원은 원심판결을 왜 파기하고 환송했나요?',
  'ANSWER': '대법원은 원심판결이 피고의 명도집행이 불법하다는 원고들의 주장사실을 인정할 충분한 자료가 없다고 판단한 것이 채증법칙에 위배되고 강제집행절차에 관한 법리를 오해한 위법이 있다고 보아 원심판결을 파기하고 환송했습니다.'},
 {'QUESTION': '주택 명도집행 시 집달관이 준수해야 할 절차는 무엇인가요?',
  'ANSWER': '주택 명도집행 시 집달관은 채무자로부터 그 점유를 회수하여 채권자에게 이전해야 하며, 이는 민사소송법 제690조 제1항에 규정되어 있습니다.'},
 {'QUESTION': '주택임대차보호법 제3조와 제12조는 해당 사건에서 어떻게 적용되었나요?',
  'ANSWER': '주택임대차보호법 제3조와 제12조는 임차인이 주민등록을 통해 대항력을 갖추었을 경우, 전세권설정등기가 말소되더라도 그 대항력을 상실하지 않는다는 점을 명시하고 있습니다.'},
 {'QUESTION': '피고가 전세권설정등기를 한 이유는 무엇인가요?',
  'ANSWER': '피고는 주택임대차보호법에 의한 임차인의 대항력을 이미 갖추었지만, 자신의 지위를 강화하기 위해 전세권설정등기를 하였다고 주장하였습니다.'},
 {'QUESTION': '대법원이 원심판결을 파기한 이유는 무엇인가요?',
  'ANSWER': '대법원은 원심이 피고의 전세권설정등기와 관련된 이유 및 경위를 충분히 심리하지 않았고, 주택임대차보호법의 법리를 오해하여 판결에 영향을 미쳤다고 판단하여 원심판결을 파기하였습니다.'},
 {'QUESTION': '여객자동차 운수사업법상 노선 여객자동차 운송사업과 전세버스 운송사업의 구별 기준은 무엇인가요?',
  'ANSWER': '여객자동차 운수사

In [17]:
import json

with open("data/jeonse_fraud", "w", encoding="utf-8") as f:
    for qa in original_qa:
        f.write(json.dumps(qa, ensure_ascii=False) + "\n")

In [18]:
import json

with open("data/jeonse_fraud.jsonl", "w", encoding="utf-8") as f:
    for qa in qa_pair:
        qa_modified = {
            "instruction": qa["QUESTION"],
            "input": "",
            "output": qa["ANSWER"],
        }
        f.write(json.dumps(qa_modified, ensure_ascii=False) + "\n")

In [26]:
#키 값 누락 있으면 사용하는 코드
import json

with open("qa_pair_judicial_precedent_data.jsonl", "w", encoding="utf-8") as f:
    for i, qa in enumerate(qa_pair):
        # 'QUESTION'과 'ANSWER' 키가 있는지 확인
        if "QUESTION" in qa and "ANSWER" in qa:
            qa_modified = {
                "instruction": qa["QUESTION"],
                "input": "",
                "output": qa["ANSWER"],
            }
            f.write(json.dumps(qa_modified, ensure_ascii=False) + "\n")
        else:
            print(f"Warning: Missing keys in item at index {i}: {qa}")


HuggingFace datasets 라이브러리를 사용하여 데이터셋을 로드합니다.


In [19]:
from datasets import load_dataset

# JSONL 파일 경로
jsonl_file = "data/jeonse_fraud.jsonl"

# JSONL 파일을 Dataset으로 로드
dataset = load_dataset("json", data_files=jsonl_file)

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 201
    })
})

In [22]:
from huggingface_hub import HfApi

# HfApi 인스턴스 생성
api = HfApi()

# 데이터셋을 업로드할 리포지토리 이름
repo_name = "ball0428/jeonse_fraud"

# 데이터셋을 허브에 푸시
dataset.push_to_hub(repo_name, token="")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ball0428/jeonse_fraud/commit/2331915d58c2465bbff95ee872d153c0a96ac73c', commit_message='Upload dataset', commit_description='', oid='2331915d58c2465bbff95ee872d153c0a96ac73c', pr_url=None, pr_revision=None, pr_num=None)